In [57]:
import pandas as pd

In [62]:
df.to_csv('df.csv')

In [61]:
df.apply(lambda x : x)

self.data[['rw_gen.month', 'rw_gen.year']] = self.data.apply(lambda x:
                                                             pd.Series(
                                                                 self.extract_date(x['rw_gen.fields.name'])),
                                                             axis=1)


,col1,col2,3
0,2,2,2
1,None,None,None
2,3,3,3


In [39]:
df['3'] = df['col1'] = df['col2']

df['col1'].astype(float)

0    2.0
1    NaN
2    3.0
Name: col1, dtype: float64

In [34]:
d = {'col1': ['1','3', None], 'col2': ['2',None, '3']}
df = pd.DataFrame(data=d)


In [9]:
globals()['In']

['',
 "d = {'col1': ['1','3', None], 'col2': ['2',None, '3']}\ndf = pd.DataFrame(data=d)\n\ndf\n\ndef a(r):\n    print(r)\n    print('**')\n    return -99\n    \ndf.col1 = df.col1.apply(lambda x : a(x), axis = 1)\n\nd['new']",
 "d = {'col1': ['1','3', None], 'col2': ['2',None, '3']}\ndf = pd.DataFrame(data=d)\n\ndf",
 'import pandas as pd',
 "d = {'col1': ['1','3', None], 'col2': ['2',None, '3']}\ndf = pd.DataFrame(data=d)\n\ndf",
 "d = {'col1': ['1','3', None], 'col2': ['2',None, '3']}\ndf = pd.DataFrame(data=d)\n\ndf",
 "d = {'col1': ['1','3', None], 'col2': ['2',None, '3']}\ndf = pd.DataFrame(data=d)\n\ndf = pd.DataFrame(df['col1'])",
 "d = {'col1': ['1','3', None], 'col2': ['2',None, '3']}\ndf = pd.DataFrame(data=d)\n\ndf = pd.DataFrame(df['col1'])\n\ndf",
 'globals()',
 "globals()['In']"]

In [25]:
df

,col1,col2
0,-99,2
1,-99,None
2,-99,3


In [50]:
class c(object):
    def __init__(self):
        self.a = 1
        
    def r(self):
        print(1)
        
    def main(self):
        self.r()

s = c()
s.main()

1


In [62]:
df = pd.DataFrame(data = {'a': [1, 2, 3], 'b': [4, 5, 6]})
ot = pd.DataFrame(data = {'a': [6, 3, 1], 'c': [14, 15, 16]})

df.merge(ot, how = 'left', left_on='a', right_on='a')

,a,b,c
0,1,4,16.0
1,2,5,NaN
2,3,6,15.0


In [121]:
'a'.

v = 'text'

v.

In [120]:
{1 : 2 if True 1 if False}

SyntaxError: invalid syntax (<ipython-input-120-2b5f64ea7908>, line 1)

In [111]:
d

['Since its emergence in April 2009, pandemic influenza A(H1N1) 2009 virus affected all parts of the world, causing the first influenza pandemic since 1968. Following the observation that 1 or 2 epidemic periods had occurred in most countries and in most WHO Regions, WHO declared on 10 Aug 2010 that the world was moving into the post-pandemic period. ([WHO, 18 Feb 2011](https://reliefweb.int/node/389128))',
 'https://reliefweb.int/disaster/ep-2009-000084-mex',
 'World',
 'wld',
 'https://api.reliefweb.int/v1/countries/254']

In [108]:
d = ['Since its emergence in April 2009, pandemic influenza A(H1N1) 2009 virus affected all parts of the world, causing the first influenza pandemic since 1968. Following the observation that 1 or 2 epidemic periods had occurred in most countries and in most WHO Regions, WHO declared on 10 Aug 2010 that the world was moving into the post-pandemic period. ([WHO, 18 Feb 2011](https://reliefweb.int/node/389128))', 'https://reliefweb.int/disaster/ep-2009-000084-mex', 'World', 'wld', 'https://api.reliefweb.int/v1/countries/254']

In [104]:
fin = json.loads(urllib.request.urlopen("https://api.reliefweb.int/v1/" \
                           "disasters?appname=vocabulary&preset=external").read().decode())['count']

fin

10

In [91]:
import urllib.request, json

with urllib.request.urlopen("https://api.reliefweb.int/v1/disasters?appname=vocabulary&preset=external&limit=10&offset=1000".format(i*1000)) as url:
    j = json.loads(url.read().decode())
    
j.keys()

dict_keys(['href', 'time', 'links', 'totalCount', 'count', 'data'])

In [56]:
import urllib.request, json

from pandas.io.json import json_normalize

r = json_normalize(json.loads(urllib.request.urlopen("https://api.hpc.tools/v1/public/global-cluster")
                                              .read().decode()))['data']

r = [v['code'] for v in r[0]]
r.remove('SHL')
s = ','.join(r)

s
# https://api.hpc.tools/v1/public/fts/flow?countryISO3=SOM&year=2015&globalClusterCode=shl,tel

'AGR,CCM,PRO-CPN,CSS,ERY,EDU,TEL,FSC,PRO-GBV,HEA,PRO-HLP,LOG,PRO-MIN,MS,NUT,OTH,PRO,WSH'

In [54]:
json_normalize(json.loads(urllib.request.urlopen("https://api.hpc.tools/v1/public/fts/flow?countryISO3=SOM&year=2015&globalClusterCode=shl,wsh")
                                              .read().decode()))

,data.flows,data.incoming.flowCount,data.incoming.fundingTotal,data.incoming.pledgeTotal,data.internal.flowCount,data.outgoing.flowCount,data.outgoing.fundingTotal,data.outgoing.pledgeTotal,meta.count,meta.language,status
0,"[{'id': '126486', 'versionId': 1, 'description...",52,38478845,0,0,6,2400471,0,58,en,ok


In [57]:
from fts_pull import fts




loaded 227 SC entries
pulled countries
num bad resps: 0


/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [59]:
from sc_pull import pull

fts(sc = pull(), test = 10, all_sector=True)

loaded 227 SC entries


TypeError: 'fts' object is not callable

In [22]:
s = pd.Series([1,2,312312414312312312312414312312414312312414312312414414312312414312312414312312414,23])

print(s.values)

[1 2
 312312414312312312312414312312414312312414312312414414312312414312312414312312414
 23]


In [4]:
json.loads(urllib.request.urlopen("https://api.hpc.tools/v1/public/location")
                                              .read().decode())

{'data': [{'adminLevel': 0,
   'id': 1,
   'iso3': 'AFG',
   'name': 'Afghanistan',
   'pcode': None},
  {'adminLevel': 0, 'id': 3, 'iso3': 'ALB', 'name': 'Albania', 'pcode': None},
  {'adminLevel': 0, 'id': 4, 'iso3': 'DZA', 'name': 'Algeria', 'pcode': None},
  {'adminLevel': 0,
   'id': 5,
   'iso3': 'ASM',
   'name': 'American Samoa',
   'pcode': None},
  {'adminLevel': 0, 'id': 6, 'iso3': 'AND', 'name': 'Andorra', 'pcode': None},
  {'adminLevel': 0, 'id': 7, 'iso3': 'AGO', 'name': 'Angola', 'pcode': None},
  {'adminLevel': 0,
   'id': 8,
   'iso3': 'AIA',
   'name': 'Anguilla (United Kingdom)',
   'pcode': None},
  {'adminLevel': 0,
   'id': 9,
   'iso3': 'ATA',
   'name': 'Antarctica',
   'pcode': None},
  {'adminLevel': 0,
   'id': 10,
   'iso3': 'ATG',
   'name': 'Antigua and Barbuda',
   'pcode': None},
  {'adminLevel': 0,
   'id': 11,
   'iso3': 'ARG',
   'name': 'Argentina',
   'pcode': None},
  {'adminLevel': 0, 'id': 12, 'iso3': 'ARM', 'name': 'Armenia', 'pcode': None},
  {

In [3]:
import urllib.request, json
import csv
import itertools
import math
import collections
import sys

import grequests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize